# 🔬 Melanoma DIP Engine - Research Testing Notebook

## ⚠️ CRITICAL MEDICAL DISCLAIMER

**THIS SOFTWARE IS FOR RESEARCH AND EDUCATIONAL PURPOSES ONLY.**

- ❌ **NOT FOR MEDICAL USE**: This tool does NOT provide medical diagnosis, risk assessment, or clinical recommendations
- ❌ **NO MEDICAL ADVICE**: Results should never be used to make medical decisions
- ❌ **RESEARCH ONLY**: All measurements and analyses are for academic research purposes
- ✅ **CONSULT PROFESSIONALS**: Always consult qualified healthcare professionals for medical assessment
- ✅ **EDUCATIONAL TOOL**: This notebook demonstrates digital image processing techniques for educational purposes

**By using this software, you acknowledge that you understand these limitations and will not use it for medical purposes.**


# Melanoma DIP Engine - Interactive Testing & Visualization

This notebook provides comprehensive testing and visualization of our Digital Image Processing pipeline for melanoma risk analysis.

## Pipeline Overview:
1. **Image Loading & Preprocessing**: Load, resize, and convert color spaces
2. **Hair Removal**: Remove artifacts using DullRazor technique
3. **Lesion Segmentation**: Extract lesion using CIELab color space analysis
4. **Feature Extraction**: Calculate ABCD/T features for clinical analysis
5. **Validation**: Compare results with ground truth using Dice coefficient


## Import Required Libraries and Modules


In [2]:
# Standard library imports
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Import our custom modules
import image_processing as ip
import feature_extraction as fe
import utils
import config

# Set matplotlib parameters for better visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ All modules imported successfully!")
print(f"📊 Image size configuration: {config.IMAGE_SIZE}")
print(f"🔧 Hair removal kernel size: {config.HAIR_REMOVAL_KERNEL_SIZE}")
print(f"🎯 Segmentation kernel size: {config.SEGMENTATION_KERNEL_SIZE}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Aakarsh Goyal\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Aakarsh Goyal\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1077, in launch_instance
    app.start()
  File "C:\Users\Aakarsh Goyal\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739,

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

## Define Sample Data Paths

**Note**: Place your sample images and ground truth masks in the `data/` folder before running this notebook.


In [ ]:
# PH2 Dataset Configuration
# Define paths to your PH2 dataset images
sample_image_path = r"C:\Users\Aakarsh Goyal\Downloads\archive\PH2Dataset\PH2 Dataset images\IMD427\IMD427_Dermoscopic_Image\IMD427.bmp"  # Dermoscopic image
ground_truth_mask_path = r"C:\Users\Aakarsh Goyal\Downloads\archive\PH2Dataset\PH2 Dataset images\IMD427\IMD427_lesion\IMD427_lesion.bmp"  # Ground truth mask

# Check if files exist
if os.path.exists(sample_image_path):
    print(f"✅ PH2 dermoscopic image found: {sample_image_path}")
    print(f"📏 File size: {os.path.getsize(sample_image_path)} bytes")
else:
    print(f"⚠️ PH2 dermoscopic image not found: {sample_image_path}")
    print("Please verify the file path is correct")

if os.path.exists(ground_truth_mask_path):
    print(f"✅ PH2 ground truth mask found: {ground_truth_mask_path}")
    print(f"📏 File size: {os.path.getsize(ground_truth_mask_path)} bytes")
else:
    print(f"⚠️ PH2 ground truth mask not found: {ground_truth_mask_path}")
    print("Please verify the file path is correct")

print("\n📊 PH2 Dataset Information:")
print("- Dermoscopic Image: High-resolution skin lesion image")
print("- Ground Truth Mask: Binary mask showing lesion region")
print("- Format: BMP (Bitmap) - already supported by our pipeline")


In [ ]:
# PH2 Dataset Preprocessing Helper Functions
def load_ph2_mask(mask_path):
    """
    Load and preprocess PH2 ground truth mask.
    PH2 masks are typically binary (black/white) and may need inversion.
    """
    # Load mask as grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if mask is None:
        raise ValueError(f"Could not load mask from {mask_path}")
    
    # Check if mask needs inversion (PH2 masks are sometimes inverted)
    # Lesion regions should be white (255), background black (0)
    lesion_pixels = np.sum(mask > 128)  # Count bright pixels
    background_pixels = np.sum(mask <= 128)  # Count dark pixels
    
    # If there are more dark pixels than bright pixels, invert the mask
    if background_pixels > lesion_pixels:
        mask = 255 - mask
        print("🔄 Inverted PH2 mask (lesion regions now white)")
    
    # Ensure binary mask (0 or 255)
    mask = np.where(mask > 128, 255, 0).astype(np.uint8)
    
    print(f"📊 PH2 mask loaded: {mask.shape}")
    print(f"🔍 Lesion pixels: {np.sum(mask > 0)}")
    print(f"📏 Lesion area: {(np.sum(mask > 0) / mask.size) * 100:.2f}% of image")
    
    return mask

def preview_ph2_images():
    """Preview PH2 images before processing"""
    print("🔍 Previewing PH2 dataset images...")
    
    # Load and preview dermoscopic image
    derm_image = cv2.imread(sample_image_path)
    if derm_image is not None:
        derm_image_rgb = cv2.cvtColor(derm_image, cv2.COLOR_BGR2RGB)
        
        # Load and preview mask
        mask = load_ph2_mask(ground_truth_mask_path)
        
        # Create preview
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(derm_image_rgb)
        axes[0].set_title('PH2 Dermoscopic Image', fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(mask, cmap='gray')
        axes[1].set_title('PH2 Ground Truth Mask', fontweight='bold')
        axes[1].axis('off')
        
        # Create overlay
        overlay = derm_image_rgb.copy()
        overlay[mask > 0] = [255, 0, 0]  # Red overlay for lesion
        axes[2].imshow(overlay)
        axes[2].set_title('Lesion Overlay', fontweight='bold')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return derm_image_rgb, mask
    else:
        print("❌ Could not load dermoscopic image")
        return None, None

# Preview the images
print("🔍 Previewing PH2 dataset images...")
preview_derm_image, preview_mask = preview_ph2_images()


In [ ]:
# Define paths to sample data
# Replace these with your actual image paths
sample_image_path = "data/sample_lesion.jpg"  # Your sample lesion image
ground_truth_mask_path = "data/sample_mask.jpg"  # Your ground truth mask

# Check if files exist
if os.path.exists(sample_image_path):
    print(f"✅ Sample image found: {sample_image_path}")
else:
    print(f"⚠️ Sample image not found: {sample_image_path}")
    print("Please place a sample lesion image in the data/ folder")

if os.path.exists(ground_truth_mask_path):
    print(f"✅ Ground truth mask found: {ground_truth_mask_path}")
else:
    print(f"⚠️ Ground truth mask not found: {ground_truth_mask_path}")
    print("Please place a ground truth mask in the data/ folder")


## Step 1: Image Loading and Preprocessing

Load the input image and perform initial preprocessing including resizing and color space conversion.


In [ ]:
# Load and preprocess the image with enhanced error handling
try:
    rgb_image, hsv_image, lab_image = ip.load_and_preprocess(sample_image_path)
    
    print("✅ Image loaded and preprocessed successfully!")
    print(f"📏 RGB Image shape: {rgb_image.shape}")
    print(f"📏 HSV Image shape: {hsv_image.shape}")
    print(f"📏 LAB Image shape: {lab_image.shape}")
    
    # Validate image quality with comprehensive checks
    quality_passed = utils.validate_image_quality(rgb_image, min_size=(200, 200))
    
    if not quality_passed:
        print("⚠️ Warning: Image quality validation failed. Results may be unreliable.")
    
except Exception as e:
    utils.create_error_report(e, "Image loading and preprocessing")
    print("Please check your image path and file format")


## Step 2: Hair Removal (DullRazor Technique)

Remove hair artifacts from the lesion image using morphological operations and inpainting.


In [ ]:
# Remove hair artifacts with advanced quality assessment
try:
    hair_free_image, hair_metrics = ip.remove_hair(rgb_image)
    
    print("✅ Hair removal completed successfully!")
    print(f"📏 Hair-free image shape: {hair_free_image.shape}")
    print(f"🔍 Hair percentage removed: {hair_metrics['hair_percentage']:.2f}%")
    print(f"🎯 Inpainting algorithm used: {hair_metrics['inpainting_algorithm']}")
    print(f"⭐ Processing complete: {hair_metrics['processing_complete']}")
    
    # Visualize before and after hair removal
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(rgb_image)
    axes[0].set_title('Original Image', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(hair_free_image)
    axes[1].set_title('After Hair Removal', fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    utils.create_error_report(e, "Hair removal processing")
    print("❌ Hair removal failed - stopping analysis for safety")
    raise Exception("Cannot continue without proper hair removal")


## Step 3: Lesion Segmentation

Segment the lesion from surrounding skin using CIELab color space analysis and morphological operations.


In [ ]:
# Segment the lesion with advanced quality assessment
try:
    binary_mask, main_contour, seg_metrics = ip.segment_lesion(hair_free_image)
    
    print("✅ Lesion segmentation completed successfully!")
    print(f"📏 Binary mask shape: {binary_mask.shape}")
    print(f"🔍 Mask area: {np.sum(binary_mask > 0)} pixels")
    print(f"🎯 Segmentation confidence: {seg_metrics['confidence_score']:.3f}")
    print(f"📊 Area percentage: {seg_metrics['area_percentage']:.2f}%")
    print(f"🔧 Segmentation method: {seg_metrics['segmentation_method']}")
    
    if main_contour is not None:
        contour_area = cv2.contourArea(main_contour)
        print(f"🔍 Contour area: {contour_area:.2f} pixels")
        print(f"✅ Contour meets minimum area requirement: {contour_area >= config.MIN_LESION_AREA}")
        
        if seg_metrics['confidence_score'] < config.MIN_CONFIDENCE_THRESHOLD:
            print("⚠️ Warning: Low segmentation confidence - results may be unreliable")
    else:
        print("⚠️ Warning: No valid contour found")
    
    # Visualize segmentation results
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(hair_free_image)
    axes[0].set_title('Hair-Free Image', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(binary_mask, cmap='gray')
    axes[1].set_title('Binary Mask', fontweight='bold')
    axes[1].axis('off')
    
    # Create overlay image
    overlay_image = utils.create_overlay_image(hair_free_image, binary_mask)
    axes[2].imshow(overlay_image)
    axes[2].set_title('Segmentation Overlay', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    utils.create_error_report(e, "Lesion segmentation")
    print("❌ Segmentation failed - stopping analysis for safety")
    raise Exception("Cannot continue without valid segmentation")


## Step 4: Feature Extraction (ABCD/T Analysis)

Extract comprehensive clinical features based on the ABCD rule and texture analysis.


In [ ]:
# Extract all features with comprehensive clinical analysis
if binary_mask is not None and main_contour is not None:
    try:
        features = fe.extract_all_features(
            original_image=hair_free_image,
            hsv_image=hsv_image,
            mask=binary_mask,
            contour=main_contour
        )
        
        print("✅ Feature extraction completed successfully!")
        print(f"📊 Total features extracted: {features.get('num_features_extracted', 0)}")
        print(f"🎯 Clinical validation: {'✅ PASSED' if features.get('clinical_validation_passed', False) else '❌ FAILED'}")
        
        # Print comprehensive research-grade feature summary
        utils.print_feature_summary(features)
        
        # Create research-focused visualization
        overlay_image = utils.create_overlay_image(hair_free_image, binary_mask)
        utils.create_clinical_visualization(
            original_img=rgb_image,
            hair_free_img=hair_free_image,
            mask=binary_mask,
            overlay=overlay_image,
            features=features,
            title="Research Analysis Report"
        )
        
        # Save analysis report
        import os
        report_path = "clinical_analysis_report.txt"
        utils.save_analysis_report(features, report_path)
        
    except Exception as e:
        utils.create_error_report(e, "Feature extraction and clinical analysis")
        print("Feature extraction failed")
        features = {'error': 'Feature extraction failed'}
        
else:
    print("❌ Cannot perform feature extraction - segmentation failed")
    features = {'error': 'Segmentation failed - no valid lesion detected'}


In [ ]:
# PH2 Dataset Validation with Ground Truth
# Compare our segmentation results with PH2 ground truth using the Dice coefficient.
if os.path.exists(ground_truth_mask_path):
    try:
        # Load PH2 ground truth mask with proper preprocessing
        gt_mask = load_ph2_mask(ground_truth_mask_path)
        
        # Resize ground truth to match our processing size
        gt_mask_resized = cv2.resize(gt_mask, config.IMAGE_SIZE, interpolation=cv2.INTER_NEAREST)
        
        # Calculate Dice coefficient
        dice_score = utils.calculate_dice_coefficient(gt_mask_resized, binary_mask)
        
        print(f"🎯 Dice Coefficient: {dice_score:.3f}")
        print(f"📊 Segmentation Quality: {'Excellent' if dice_score > 0.8 else 'Good' if dice_score > 0.6 else 'Needs Improvement'}")
        
        # Visualize comparison
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(gt_mask_resized, cmap='gray')
        axes[0].set_title('PH2 Ground Truth Mask', fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(binary_mask, cmap='gray')
        axes[1].set_title('Our Segmentation', fontweight='bold')
        axes[1].axis('off')
        
        # Create difference visualization
        difference = cv2.absdiff(gt_mask_resized, binary_mask)
        axes[2].imshow(difference, cmap='hot')
        axes[2].set_title(f'Difference (Dice: {dice_score:.3f})', fontweight='bold')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Calculate additional metrics
        intersection = np.sum((gt_mask_resized > 0) & (binary_mask > 0))
        union = np.sum((gt_mask_resized > 0) | (binary_mask > 0))
        iou = intersection / union if union > 0 else 0
        
        print(f"📊 Additional Metrics:")
        print(f"   - Intersection over Union (IoU): {iou:.3f}")
        print(f"   - Ground Truth Area: {np.sum(gt_mask_resized > 0)} pixels")
        print(f"   - Our Segmentation Area: {np.sum(binary_mask > 0)} pixels")
        print(f"   - Area Difference: {abs(np.sum(gt_mask_resized > 0) - np.sum(binary_mask > 0))} pixels")
        
    except Exception as e:
        print(f"❌ Error in PH2 ground truth validation: {e}")
        
else:
    print("⚠️ PH2 ground truth mask not available - skipping validation")


## Step 5: Validation with Ground Truth (Optional)

Compare our segmentation results with ground truth using the Dice coefficient.


In [ ]:
# Load and validate against ground truth
if os.path.exists(ground_truth_mask_path):
    try:
        # Load ground truth mask
        gt_mask = cv2.imread(ground_truth_mask_path, cv2.IMREAD_GRAYSCALE)
        
        if gt_mask is not None:
            # Resize ground truth to match our processing size
            gt_mask_resized = cv2.resize(gt_mask, config.IMAGE_SIZE, interpolation=cv2.INTER_NEAREST)
            
            # Calculate Dice coefficient
            dice_score = utils.calculate_dice_coefficient(gt_mask_resized, binary_mask)
            
            print(f"🎯 Dice Coefficient: {dice_score:.3f}")
            print(f"📊 Segmentation Quality: {'Excellent' if dice_score > 0.8 else 'Good' if dice_score > 0.6 else 'Needs Improvement'}")
            
            # Visualize comparison
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            axes[0].imshow(gt_mask_resized, cmap='gray')
            axes[0].set_title('Ground Truth Mask', fontweight='bold')
            axes[0].axis('off')
            
            axes[1].imshow(binary_mask, cmap='gray')
            axes[1].set_title('Our Segmentation', fontweight='bold')
            axes[1].axis('off')
            
            # Create difference visualization
            difference = cv2.absdiff(gt_mask_resized, binary_mask)
            axes[2].imshow(difference, cmap='hot')
            axes[2].set_title(f'Difference (Dice: {dice_score:.3f})', fontweight='bold')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
        else:
            print("❌ Could not load ground truth mask")
            
    except Exception as e:
        print(f"❌ Error in ground truth validation: {e}")
        
else:
    print("⚠️ Ground truth mask not available - skipping validation")


## Complete Pipeline Visualization

Display the entire pipeline in a single comprehensive visualization.


In [ ]:
# Create comprehensive pipeline visualization
try:
    overlay_image = utils.create_overlay_image(hair_free_image, binary_mask)
    
    utils.visualize_steps(
        original_img=rgb_image,
        hair_free_img=hair_free_image,
        mask=binary_mask,
        overlay=overlay_image,
        title="Complete Melanoma DIP Pipeline"
    )
    
    print("🎉 Complete pipeline visualization generated successfully!")
    
except Exception as e:
    print(f"❌ Error in pipeline visualization: {e}")


## Summary and Next Steps

This notebook demonstrates a complete DIP pipeline for melanoma analysis. The extracted features can be used for:

1. **Clinical Decision Support**: ABCD rule assessment for dermatologists
2. **Machine Learning**: Feature vectors for classification models
3. **Research**: Objective quantification of lesion characteristics
4. **Monitoring**: Longitudinal tracking of lesion changes

### To improve the pipeline:
- Add more sophisticated hair detection algorithms
- Implement multi-scale segmentation approaches
- Include additional texture features (LBP, Gabor filters)
- Add diameter calculation from pixel measurements
- Implement lesion tracking across multiple images

### Key Features Extracted:
- **Asymmetry**: Shape symmetry score (0.0-1.0)
- **Border Irregularity**: Compactness index (>1.0)
- **Color Variation**: Number of distinct colors (≥1)
- **Texture Contrast**: GLCM contrast measure
- **Texture Homogeneity**: GLCM homogeneity measure

### Usage Instructions:
1. Place sample images in the `data/` folder
2. Update the file paths in the notebook
3. Run each cell sequentially
4. Analyze the results and features extracted
5. Use the Dice coefficient to validate segmentation quality
